In [1]:
# GROUP: Graphkrone
# Members: Marcello Negri     19-945-450
#          Riccardo Uslenghi  19-954-262

import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

np.set_printoptions(precision=4, suppress=True, linewidth=1000)
out_file = open("solutions.txt", "w")

# loading dataset from 'train.csv', skiping first line (labels)
dataset = np.loadtxt('train.csv', delimiter=",", skiprows=1)

Y = dataset[:,1]
X = dataset[:,2:]

X = np.c_[X, np.square(X[:,:5])]
X = np.c_[X, np.exp(X[:,:5])]
X = np.c_[X, np.cos(X[:,:5])]
X = np.c_[X, np.ones((len(Y),1))]

# performing k-fold with 10 folds for each lambda in the range
kf = KFold(n_splits=10)
lambda_range = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000]

print('\nWe used cross validation with 10 fold k-fold to pick the optimal lambda')
print('We explored two regularisers: Ridge and Lasso')

average = []
print('\nlambda \t\t avg RMSE: Lasso \t avg RMSE: Ridge')
for l in lambda_range:
    rmse_lasso = []
    rmse_ridge = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        lasso = Lasso(alpha=l, fit_intercept=False, max_iter=50000)
        lasso.fit(X_train,Y_train)
        ridge = Ridge(alpha=l, fit_intercept=False)
        ridge.fit(X_train,Y_train)
        RMSE_lasso = np.sqrt(mean_squared_error(Y_test, lasso.predict(X_test)))
        RMSE_ridge = np.sqrt(mean_squared_error(Y_test, ridge.predict(X_test)))
        rmse_lasso.append(RMSE_lasso)
        rmse_ridge.append(RMSE_ridge)
    avg_lasso = sum(rmse_lasso)/len(rmse_lasso)
    avg_ridge = sum(rmse_ridge)/len(rmse_ridge)
    print('{0} \t --> \t {1} \t {2}'.format(l, avg_lasso, avg_ridge))
    average.append(avg_lasso)
    average.append(avg_ridge)

best_avg = min(average)
index = average.index(best_avg)

# model selection for regulariser:
# Ridge or Lasso chosen with lambda that returns lowest average of rmse

if index%2 == 0:
    best_alpha_lasso = lambda_range[int(index/2)]
    print('\nThe best regulariser is Lasso: lambda = {0} with avg RMSE = {1}\n'\
    .format(best_alpha_lasso, best_avg))
    lasso = Lasso(fit_intercept=False, alpha=best_alpha_lasso, max_iter=50000)
    lasso.fit(X, Y)
    coeff = lasso.coef_
    RMSE_final = np.sqrt(mean_squared_error(Y, lasso.predict(X)))
    print(" THIS IS YOUR GRADE : ", RMSE_final )

else:
    best_alpha_ridge = lambda_range[int(index/2)]
    print('\nThe best regulariser is Ridge: lambda = {0} with avg RMSE = {1}\n'\
    .format(best_alpha_ridge, best_avg))
    ridge = Ridge(fit_intercept=False, alpha=best_alpha_ridge)
    ridge.fit(X, Y)
    coeff = ridge.coef_

for i in coeff:
    out_file.write('{}\n'.format(i))
out_file.close()



We used cross validation with 10 fold k-fold to pick the optimal lambda
We explored two regularisers: Ridge and Lasso

lambda 		 avg RMSE: Lasso 	 avg RMSE: Ridge
0.001 	 --> 	 4.930398480617299 	 4.935154747723333
0.005 	 --> 	 4.911438189986663 	 4.935038423416488
0.01 	 --> 	 4.908269296115286 	 4.93481698164208
0.05 	 --> 	 4.897560840755881 	 4.933126938185666
0.1 	 --> 	 4.900289437944378 	 4.931022291773101
0.5 	 --> 	 4.931843855845708 	 4.9201267635744825
1 	 --> 	 4.995820363642616 	 4.913839469009035
5 	 --> 	 4.997930834936108 	 4.901743842918328
10 	 --> 	 4.997930834936108 	 4.898702597747051
50 	 --> 	 4.997930834936108 	 4.898429656048128
100 	 --> 	 4.997930834936108 	 4.900127570603312
500 	 --> 	 4.997930834936108 	 4.903101282546723
1000 	 --> 	 4.997930834936108 	 4.9044119212549395

The best regulariser is Lasso: lambda = 0.05 with avg RMSE = 4.897560840755881

 THIS IS YOUR GRADE :  4.9029150291730055
